## Ideal Gas

To generate a mixture, first it is necessary to create the Substance objects 
that will be in the mixture. This could be defined by the user or with the
Thermo library. For instance:

In [1]:
import reactord as rd

so2 = rd.Substance.from_thermo_database("sulfur dioxide")
o2 = rd.Substance.from_thermo_database("oxygen")
so3 = rd.Substance.from_thermo_database("sulfur trioxide")
co2 = rd.Substance.from_thermo_database("carbon dioxide")

Now we create a mixture, introducing the objects as a dictionary. 
In this case, an instance of the IdealGas class is created:

In [2]:
gas_mixture = rd.mix.IdealGas(A=so2, B=o2, C=so3, Inert=co2)

# The mixture is a list of substances to which we can access:
print(gas_mixture.substances[0].normal_boiling_point)
print(gas_mixture.substances[1].name)
print(gas_mixture.substances[2].critical_pressure)

263.1
oxygen
8207325.0


The list of methods that an IdealGas have are the following:

Methods inherited from AbstractMix.  
- mol_fractions  
- mixture_molecular_weight  
- partial_pressures  
- molar_density  
- mass_density  

Methods implemented by IdealGas.  
- volume  
- concentrations  
- mix_heat_capacity  
- formation_enthalpies_correction  
- mixture_viscosity  

The methods inherited from AbstractMix are explained in the [AbstractMix tutorial](abstract_mix.ipynb). On the other hand, each method implemented by IdealGas 
may require that the substances have specific attributes or methods defined. 

__volume(moles, temperature, pressure)__

The volume method recieves the moles of each substances [mol], temperature [K] 
and pressure [Pa], and return the volume of the mix [m³]. 
The method uses the ideal gas law as:

$$V = \frac{nRT}{P}$$


In [3]:
volume = gas_mixture.volume([5, 10, 15, 20], 700, 101325)

print(rf"{volume} m³")

2.872007812833589 m³


__concentrations(moles, temperature, pressure)__

The concentrations method recieves the moles [mol], temperature [K] and 
pressure [Pa] and returns the concentration of each mix's substance [mol/m³].
The method uses the ideal gas law to evaluate the mixture density as:

$$\rho_{mix} = \frac{P}{RT}$$

The concentration of each substance is evaluated from the mixture's density and
the the substance's molar fractions.

$$C_{i} = z_{i}\rho_{mix}$$


In [4]:
concentrations = gas_mixture.concentrations([5, 10, 15, 20], 700, 101325)

print(rf"{concentrations} mol/m³")

[1.74094234 3.48188468 5.22282702 6.96376936] mol/m³


__mix_heat_capacity(moles, temperature, pressure)__

The mix_heat_capacity method recieves the moles [mol], temperature [K] and 
pressure [Pa] and returns the mix's heat capacity [J/mol/K]. **The method 
requires that each substance have the ideal gas heat capacity method 
(${C_{p}}_{i}$) defined**. The mixture's heat capacity is calculated from the 
mole fractions and the pure ideal gas heat capacity of each substance:

$${C_{p}}_{mix} = \sum z_{i} {C_{p}}_{i}$$

The method is used by non isothermal reactors.

In [5]:
heat_capacity = gas_mixture.mix_heat_capacity([5, 10, 15, 20], 700, 101325)

print(rf"{heat_capacity} J/mol/K")

52.91656909879653 J/mol/K


__formation_enthalpies_correction(temperature, pressure)__

The formation_enthalpies_correction method recieves the temperature [K] and 
pressure [Pa] and returns the enthalpy of formation of each substance [J/mol/K]
at the specified temperature and pressure. **The method requires that each 
substance have the ideal gas heat capacity (${{C_{p}}_{i}}_{(T,P)}$) method and 
the ideal gas enthalpy of formation (${\Delta_{f}H_{i}^{298.15}}$) defined**.

The corrected formation enthalpy of a substance is:
$${\Delta_{f}H}_{i} = {\Delta_{f}H_{i}^{298.15}} + \int_{298.15}^{T}{{C_{p}}_{i}}_{(T,P)}dT$$

The method return the term:

$$\int_{298.15}^{T}{{C_{p}}_{i}}_{(T,P)}dT$$

for each substance.

The method is used by non isothermal reactors.

In [6]:
ideal_gas_std_formation_enthalpies = [
    substance.formation_enthalpy_ig for substance in gas_mixture.substances
]

formation_enthalpies = gas_mixture.formation_enthalpies_correction(700, 101325)

print("Ideal Gas formation Enthalpies:")
print(rf"{ideal_gas_std_formation_enthalpies} J/mol/K")

print("Corrected formation Enthalpies:")
print(rf"{formation_enthalpies} J/mol/K")

Ideal Gas formation Enthalpies:
[-296800.0, 0.0, -395700.0, -393474.0] J/mol/K
Corrected formation Enthalpies:
[18700.93185285 12498.74118894 25208.0487025  17752.76914702] J/mol/K


__mixture_viscosity(moles, temperature, pressure)__

The mixture_viscosity method recieves the moles[mol], temperature [K] and 
pressure [Pa] and returns the mixture's viscocity [Pa s] at the specified 
temperature and pressure. **The method requires that each 
substance have the gas viscocity ($\mu_{i}$) and molecular weight ($Mw_{i}$) 
defined**. For the calculation, the Herning_Zipperer method from the Thermo 
library is used.

$$\mu = \frac{\sum z_{i}\mu_{i}\sqrt{Mw_{i}}}{\sum z_{i}\sqrt{Mw_{i}}}$$

The method is used by non isobaric flow reactors.


In [7]:
viscosity = gas_mixture.mixture_viscosity([5, 10, 15, 20], 700, 101325)

print(rf"{viscosity} Pa s")

3.119952701239795e-05 Pa s
